In [ ]:
# パブリック・インターネット・アクセスを有効にします
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"

In [ ]:
# pip自体をアップデートさせます
!pip install -qU --upgrade pip

In [ ]:
# 必要なライブラリをまとめてインストールします
!pip install -qU langchain langchain-community pypdf datapane==0.17.0 pandas==1.4.0 oracledb oci oci-cli 

# まとめてインストールできない場合(依存関係に問題がある場合)は、個別にインストールします
# pip install -qU langchain
# pip install -qU langchain-community
# pip install -qU pypdf
# pip install -qU sktime==0.24.0
# pip install -qU datapane==0.17.0
# pip install -qU pandas==1.4.0
# pip install -qU oracledb
# pip install -qU oci
# pip install -qU oci-cli

In [ ]:
import os

# PDFが存在するか確認します
pdf_path = "/home/datascience/tmp/SS_CEO_Message.pdf"
if os.path.exists(pdf_path):
    print("PDF exists")
else:
    print("PDF does not exist")

In [ ]:
import oracledb

# 利用するBaseDBに接続します。接続が成功した場合は、「Connection successful!」と出力されます
oracledb.init_oracle_client()

username = "docuser"
password = "WelCome123#123#"
dsn = "<BaseDBのパブリックIPアドレス>:1521/<PDBの接続文字列>"

try:
    connection = oracledb.connect(user=username, password=password, dsn=dsn)
    print("Connection successful!")
except oracledb.DatabaseError as e:
    error, = e.args
    print(f"Database connection faild: {error.code} - {error.masege}")
except Exception as e:
    print(f"Connection failed: {str(e)}")

In [ ]:
from langchain.document_loaders import PyPDFLoader

# 利用するPDFファイルのテキストをロードします
loader = PyPDFLoader("/home/datascience/tmp/SS_CEO_Message.pdf")
documents1 = loader.load_and_split()
print(documents1)

In [ ]:
from langchain.text_splitter import CharacterTextSplitter

# テキストを任意のサイズに分割します
text_splitter = CharacterTextSplitter(separator="。", chunk_size=200, chunk_overlap=10)
docs1 = text_splitter.split_documents(documents1)
print(docs1)

In [ ]:
import pandas as pd

# 分割したテキストを整理します
contents = []
for doc in docs1:
    contents.append(doc.page_content)
pd.DataFrame(contents)

In [ ]:
# 必要なライブラリをimportします
from langchain_community.vectorstores import oraclevs
from langchain_community.vectorstores.oraclevs import OracleVS
from langchain_community.vectorstores.utils import DistanceStrategy
from langchain_core.documents import Document
from langchain_community.embeddings import OCIGenAIEmbeddings

In [ ]:
# 分割したテキストを埋め込みモデル(OCI Generative AI Serviceのembed-multilingual-v3.0)でベクトルに変換します
embeddings = OCIGenAIEmbeddings(
    model_id="cohere.embed-multilingual-v3.0",
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="<コンパートメントOCID>"
)

In [ ]:
# ベクトル変換したテキストをベクトル・データベースにロードします
vector_store_dot = OracleVS.from_documents(
    docs1, # 分割済みのテキスト
    embeddings, # 定義済の埋め込みモデル
    client=connection, # 利用するBaseDBデータベースとのconnectionオブジェクト
    table_name="doc_table_ss", # 新規作成する表の名前を指定
    distance_strategy=DistanceStrategy.DOT_PRODUCT, # ベクトル検索時に使う距離計算の方法
)

In [ ]:
# 索引を作成します
oraclevs.create_index(connection, vector_store_dot, params={"idx_name": "SS", "idx_type": "IVF"})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# プロンプトのテンプレートを作成します
template = """contextに従って回答してください:
{context}

質問: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain_community.chat_models.oci_generative_ai import ChatOCIGenAI

# テキスト生成モデルを指定します
llm = ChatOCIGenAI(
    model_id="cohere.command-r-plus", # 他のモデルでも可能
    service_endpoint="https://inference.generativeai.us-chicago-1.oci.oraclecloud.com",
    compartment_id="<コンパートメントOCID>",
    model_kwargs={"temperature": 0.7, "max_tokens": 500},
)

In [ ]:
# ベクトル検索を実行するベクトルデータベースを定義します
retriever = vector_store_dot.as_retriever()

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 今まで定義したものをChain(RAG構成)として繋ぎます
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
# 定義したChain(RAG構成)を利用して、問い合わせます
print(chain.invoke("スマートスタイルの社長は誰ですか?"))
print(chain.invoke("スマートスタイルの創業はいつですか?"))
print(chain.invoke("スマートスタイルの専門性は何ですか?"))
print(chain.invoke("スマートスタイルのコーポレートビジョンはなんですか?"))

In [ ]:
# Chainを利用しない(非RAG構成)で問い合わせます
response1 = llm.invoke("スマートスタイルの社長は誰ですか?")
print(response1.content) 
response2 = llm.invoke("スマートスタイルの創業はいつですか?")
print(response2.content) 
response3 = llm.invoke("スマートスタイルの専門性は何ですか?")
print(response3.content) 
response4 = llm.invoke("スマートスタイルのコーポレートビジョンはなんですか?")
print(response4.content) 